Cette page doit être lancée sur Kaggle, depuis la compétition pour avoir accès aux données. Sinon vous devez récupérer les données séparément.

In [1]:
import tensorflow as tf
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.display import FileLink
SEED = 25
np.random.seed(SEED)
tf.random.set_seed(SEED)
OUTPUT_DIR = '/kaggle/working/outputs'

In [2]:
!mkdir /kaggle/working/outputs

## Lire les images

cf https://keras.io/api/data_loading/image/ et 
https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [3]:
directory_path = "/kaggle/input/navires-2023-la-mano/ships16x24/ships_16x24_10cat/data"
ds_train, ds_val = tf.keras.utils.image_dataset_from_directory(directory_path, labels='inferred',subset='both',shuffle=True,validation_split=0.2,seed=SEED,image_size=(16,24))


Found 35515 files belonging to 10 classes.
Using 28412 files for training.
Using 7103 files for validation.


In [5]:
ds_train

<BatchDataset element_spec=(TensorSpec(shape=(None, 16, 24, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

## Résultat à soumettre

In [8]:
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D,MaxPooling2D, MaxPooling2D, Dropout,Flatten, Dense, BatchNormalization

def build_model() -> Model:
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Rescaling(1./255, input_shape=(16, 24, 3)),
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu',padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu',padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.25),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                metrics=['accuracy'])
    
    return model
model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 16, 24, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 14, 22, 32)        896       
                                                                 
 batch_normalization (BatchN  (None, 14, 22, 32)       128       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 7, 11, 32)        0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 7, 11, 32)         0         
                                                                 
 conv2d_1 (Conv2D)           (None, 7, 11, 64)         1

In [9]:
def test_phase(model):
    file_name = f'reco_nav_{model.name}.csv'
    X_test = np.load('/kaggle/input/navires-2023-la-mano/test.npy', allow_pickle=True)
    X_test = X_test.astype('float32')
    res = model.predict(X_test).argmax(axis=1)
    df = pd.DataFrame({"Category":res})
    df.to_csv(os.path.join(OUTPUT_DIR,file_name), index_label="Id")

def save_history(model,history):
    history_df = pd.DataFrame(history.history)
    hist_file_name = f"history_{model.name}.csv"
    history_df.to_csv(os.path.join(OUTPUT_DIR,hist_file_name))

def save_model(model):
    model_file_name = f"{model.name}.h5"
    model.save(os.path.join(OUTPUT_DIR,model_file_name))

In [14]:
class CheckValidationLossCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, patience=0):
        self.patience = patience
        self.counter = 0

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        train_loss = logs.get('loss')
        if val_loss < train_loss:
            self.counter = 0
            print(f"\nValidation loss ({val_loss:.4f}) is less than training loss ({train_loss:.4f})")
        else:
            self.counter +=1
            
        if(self.counter == self.patience):
            self.model.stop_training = True

In [15]:
def train_and_test(epochs=50,model=None):
    callback = CheckValidationLossCallback(patience=3)#tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
    if model is None:
        model = build_model() 
    history = model.fit(ds_train,epochs=epochs,validation_data=ds_val,callbacks=[callback])
    test_phase(model)
    save_model(model)


train_and_test(epochs=75,model=model)

Epoch 1/75
887/888 [============================>.] - ETA: 0s - loss: 1.7870 - accuracy: 0.3779
Validation loss (1.6029) is less than training loss (1.7868)
888/888 [==============================] - 27s 31ms/step - loss: 1.7868 - accuracy: 0.3780 - val_loss: 1.6029 - val_accuracy: 0.4440
Epoch 2/75
887/888 [============================>.] - ETA: 0s - loss: 1.7038 - accuracy: 0.4064
Validation loss (1.4804) is less than training loss (1.7036)
888/888 [==============================] - 27s 30ms/step - loss: 1.7036 - accuracy: 0.4065 - val_loss: 1.4804 - val_accuracy: 0.4888
Epoch 3/75
886/888 [============================>.] - ETA: 0s - loss: 1.6416 - accuracy: 0.4320
Validation loss (1.4279) is less than training loss (1.6418)
888/888 [==============================] - 27s 31ms/step - loss: 1.6418 - accuracy: 0.4319 - val_loss: 1.4279 - val_accuracy: 0.4998
Epoch 4/75
887/888 [============================>.] - ETA: 0s - loss: 1.5921 - accuracy: 0.4481
Validation loss (1.3941) is less t

In [16]:
! zip -r  `date +%m%d%H%M`_result.zip outputs

  adding: outputs/ (stored 0%)
  adding: outputs/reco_nav_sequential.csv (deflated 61%)
  adding: outputs/sequential.h5 (deflated 12%)


In [10]:
!rm -rf ships32/

In [11]:
print("Nombre de couches : ", len(model.layers))

Nombre de couches :  17
